<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/solver_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip show qiskit
#!pip install qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

**Possibly need to restart runtime here.**

In [2]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 100 (delta 44), reused 20 (delta 10), pack-reused 0
Receiving objects: 100% (100/100), 197.66 KiB | 8.59 MiB/s, done.
Resolving deltas: 100% (44/44), done.


# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#./Summer2021/intermediateScenario
#./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [2]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [3]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [4]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [5]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [6]:
exec(open("./model.py").read())

* building wado model
model building done in 0.07520079612731934 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [7]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [8]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [9]:
model

docplex.mp.Model['docplex_model1']

In [10]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [11]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


#Minimum Eigen Optimizer example

In [12]:
#https://quantum-computing.ibm.com/lab/docs/iql/optimization

from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/

In [13]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [14]:
%time exact_result = exact.solve(qubo)
print(exact_result)

CPU times: user 30 ms, sys: 207 µs, total: 30.2 ms
Wall time: 28.9 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [15]:
op, offset = qubo.to_ising()
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)

In [16]:
%time exact_result = exact.solve(qp)
print(exact_result)

CPU times: user 19.5 ms, sys: 327 µs, total: 19.8 ms
Wall time: 21.4 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [17]:
%time qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.variational_forms')


CPU times: user 386 ms, sys: 44.6 ms, total: 430 ms
Wall time: 444 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [18]:
%time rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

CPU times: user 273 ms, sys: 19.5 ms, total: 293 ms
Wall time: 274 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


#Try Grover Optimizer

In [19]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model

backend = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)


In [20]:
%time results = grover_optimizer.solve(qp)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/amplitude_amplifiers/grover.py:216: DeprecationWarning: The package qiskit.aqua.algorithms.amplitude_amplifiers is deprecated. It was moved/refactored to qiskit.algorithms.amplitude_amplifiers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.amplitude_amplifiers', 'qiskit-terra')


CPU times: user 53.6 s, sys: 2.2 s, total: 55.8 s
Wall time: 51.8 s
x=[0. 1. 0.]
fval=-2.0


In [21]:
%time results = grover_optimizer.solve(qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

CPU times: user 59.8 s, sys: 2.35 s, total: 1min 2s
Wall time: 57.8 s
x=[0. 1. 0.]
fval=-2.0


#Try ADMM Optimizer


In [53]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt 

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

# If CPLEX is installed, you can uncomment this line to import the CplexOptimizer.
# CPLEX can be used in this tutorial to solve the convex continuous problem, 
# but also as a reference to solve the QUBO, or even the full problem.
# 
from qiskit.optimization.algorithms import CplexOptimizer

In [54]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator')))

# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs


In [55]:
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [56]:
admm_params = ADMMParameters(
                            rho_initial=1001, 
                            beta=1000, 
                            factor_c=900,
                            maxiter=100,
                            three_block=True, tol=1.e-6
                        )

In [67]:
# in case CPLEX is installed it can also be used for the convex problems, the QUBO, 
# or as a benchmark for the full problem.
#
cplex = CplexOptimizer()

# define QUBO optimizer
# qubo_optimizer = exact #qaoa #rqaoa #grover_optimizer

# qubo_optimizer = cplex  # uncomment to use CPLEX instead

i=-1
optms=['cplex','exact','qaoa','rqaoa']
for qubo_optimizer in [cplex,exact,qaoa,rqaoa]:

  # define classical optimizer
  #convex_optimizer = cobyla
  convex_optimizer = cplex  # uncomment to use CPLEX instead

  # initialize ADMM with classical QUBO and convex optimizer
  admm = ADMMOptimizer(params=admm_params, 
                      qubo_optimizer=qubo_optimizer,
                      continuous_optimizer=convex_optimizer)

  # run ADMM to solve problem
  i+=1
  print("\nOptimizer:",optms[i])
  %time result = admm.solve(qubo)
  print("x={}".format(result.x))
  print("fval={:.2f}".format(result.fval))


Optimizer: cplex
CPU times: user 108 ms, sys: 139 ms, total: 246 ms
Wall time: 373 ms
x=[0. 0. 0.]
fval=0.00

Optimizer: exact
CPU times: user 430 ms, sys: 136 ms, total: 566 ms
Wall time: 678 ms
x=[0. 0. 0.]
fval=0.00

Optimizer: qaoa
CPU times: user 583 ms, sys: 125 ms, total: 708 ms
Wall time: 779 ms
x=[0. 0. 0.]
fval=0.00

Optimizer: rqaoa
CPU times: user 4.31 s, sys: 892 ms, total: 5.2 s
Wall time: 5.9 s
x=[0. 1. 0.]
fval=-2.00


In [68]:
# run ADMM to solve problem
%time result = admm.solve(qp)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

CPU times: user 1.49 s, sys: 356 ms, total: 1.85 s
Wall time: 2.17 s
x=[0. 1. 0.]
fval=-2.00


In [69]:
%time result = admm.solve(qubo)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

CPU times: user 4.52 s, sys: 1.05 s, total: 5.56 s
Wall time: 6.32 s
x=[0. 1. 0.]
fval=-2.00


# 3. Solve the problem using QISKit

In [70]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [71]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -496.0, 3: -464.0, 4: -480.0, 5: -496.0}
linear array:			 [-464. -480. -496. -464. -480. -496.    0.    0.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-496.0
  (0, 3)	-464.0
  (0, 4)	-480.0
  (0, 5)	-496.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

In [72]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)


In [73]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')

print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

constant:			 1.25064e+28
linear array as sparse matrix:
   (0, 0)	-9.216e+25
  (0, 1)	-1.8432e+26
  (0, 2)	-3.6864e+26
  (0, 3)	-7.3728e+26
  (0, 4)	-1.47456e+27
  (0, 5)	-2.94912e+27
  (0, 6)	-5.89824e+27
  (0, 7)	-1.179648e+28
  (0, 8)	-2.359296e+28
  (0, 9)	-4.718592e+28
  (0, 10)	-9.437184e+28
  (0, 11)	-1.8874368e+29
  (0, 12)	-3.7748736e+29
  (0, 13)	-7.5497472e+29
  (0, 14)	-1.50994944e+30
  (0, 15)	-3.01989888e+30
  (0, 16)	-6.03979776e+30
  (0, 17)	-1.207959552e+31
  (0, 18)	-2.415919104e+31
  (0, 19)	-4.831838208e+31
  (0, 20)	-9.663676416e+31
  (0, 21)	-1.9327352832e+32
  (0, 22)	-3.8654705664e+32
  (0, 23)	-7.7309411328e+32
  (0, 24)	-1.54618822656e+33
  :	:
  (0, 869)	-6.912e+25
  (0, 870)	-1.3824e+26
  (0, 871)	-2.7648e+26
  (0, 872)	-5.5296e+26
  (0, 873)	-1.00224e+27
  (0, 874)	-2.592e+25
  (0, 875)	-5.184e+25
  (0, 876)	-1.0368e+26
  (0, 877)	-2.0736e+26
  (0, 878)	-4.1472e+26
  (0, 879)	-3.6288e+26
  (0, 880)	-5.76e+25
  (0, 881)	-1.152e+26
  (0, 882)	-2.304e+26
  (0,

In [ ]:
op, offset = mod_Qubo.to_ising()
QPI=QuadraticProgram()
QPI.from_ising(op, offset, linear=True)

In [ ]:
print('constant:\t\t\t', QPI.objective.constant)
print('linear array as sparse matrix:\n', QPI.objective.linear.coefficients, '\n')

print('quadratic matrix as sparse matrix:\n', QPI.objective.quadratic.coefficients)

In [ ]:
#exact, qaoa, rqaoa, grover_optimizer, admm
%time result = admm.solve(mod_Qubo)

print(result)

#Minimum Eigen Optimizer using VQE

In [20]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.operators import PauliExpectation, CVaRExpectation
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit import execute, Aer
from qiskit.aqua import aqua_globals

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

In [22]:
aqua_globals.random_seed = 123456
n = 100            # number of assets for the variational ansatz

In [24]:
# set classical optimizer
maxiter = 100
optimizer = COBYLA(maxiter=maxiter)

# set variational ansatz
var_form = RealAmplitudes(n, reps=1)
m = var_form.num_parameters

# set backend
backend_name = 'qasm_simulator'  # use this for QASM simulator
# backend_name = 'statevector_simulator'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)  

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25]  # confidence levels to be evaluated

In [ ]:
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [-(obj + offset)]  

# loop over all given alpha values
for alpha in alphas:
    
    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
    cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373
    
    # initialize VQE using CVaR
    vqe = VQE(expectation=cvar_exp, optimizer=optimizer, var_form=var_form, quantum_instance=backend,
              callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha))
   
    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(vqe)

    # solve problem
    %time results[alpha] = opt_alg.solve(mod_Qubo) #VGG qp, mod, mod2, mod_Qubo
    
    # print results
    print('alpha = {}:'.format(alpha))
    print(results[alpha])
    print()